In [1]:
using InMemoryDatasets
import InMemoryDatasets as IMD
using BenchmarkTools
using TimerOutputs
using DLMReader
#using Revise
using Random

In [3]:
include("../src/main.jl")

cartesianjoin_timer (generic function with 1 method)

# Data

In [2]:
Random.seed!(792)
@show dsl = Dataset(xid = [111,222,333,444,222], 
                    x1 = [1,2,1,4,3], 
                    x2 = [-1.2,-3,2.1,-3.5,2.8],
                    #x3 = [Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-04"), Date("2019-10-03"), Date("2019-10-03")],
                    x4 = ["abcd","efgh","ijkl","mnop","qrst"]);

@show dsr = Dataset(yid = [111,111,222,444,333],
                    y1 = [3,3,3,3,3],
                    y2 = [0,0, -3,1,2],
                    #y3 = [Date("2019-10-01"),Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05"), Date("2019-10-05")],
                    y4 = ["abc","abcd","efg","mnop","qrst"]);

dsl_big = Dataset(rand(10^3, 5), :auto);
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
modify!(dsl_big,[:x3] .=> byrow(string));

dsr_big = Dataset(rand(10^3, 5), :auto);
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
modify!(dsr_big,[:x3] .=> byrow(string));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big);

dsl_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
modify!(dsl_big2,[:x3] .=> byrow(string));

dsr_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
modify!(dsr_big2,[:x3] .=> byrow(string));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big2);

dsl = Dataset(xid = [111, 222, 333, 444, 222], x1 = [1, 2, 1, 4, 3], x2 = [-1.2, -3, 2.1, -3.5, 2.8], x4 = ["abcd", "efgh", "ijkl", "mnop", "qrst"]) = 5×4 Dataset
 Row │ xid       x1        x2        x4
     │ identity  identity  identity  identity
     │ Int64?    Int64?    Float64?  String?
─────┼────────────────────────────────────────
   1 │      111         1      -1.2  abcd
   2 │      222         2      -3.0  efgh
   3 │      333         1       2.1  ijkl
   4 │      444         4      -3.5  mnop
   5 │      222         3       2.8  qrst
dsr = Dataset(yid = [111, 111, 222, 444, 333], y1 = [3, 3, 3, 3, 3], y2 = [0, 0, -3, 1, 2], y4 = ["abc", "abcd", "efg", "mnop", "qrst"]) = 5×4 Dataset
 Row │ yid       y1        y2        y4
     │ identity  identity  identity  identity
     │ Int64?    Int64?    Int64?    String?
─────┼────────────────────────────────────────
   1 │      111         3         0  abc
   2 │      111         3         0  abcd
   3 │      222         3        -3  

In [44]:
filewriter("../files/dsl_big.csv", dsl_big)
filewriter("../files/dsr_big.csv", dsr_big)
filewriter("../files/dsl_big2.csv", dsl_big2)
filewriter("../files/dsr_big2.csv", dsr_big2)

# Time & allocations

## functions for join

In [4]:
function fun1(x,y) 
  x <= y
end

function fun2(x,y) 
  x >= y
end

function fun3(x,y) 
  length(x) == length(y)
end

fun3 (generic function with 1 method)

In [43]:
Threads.nthreads()

8

## small example

In [6]:
newds = cartesianjoin(dsl,dsr,on = [:xid=>:yid, :x1=>:y1=>fun1],
            multiple_match=[false,false],
            obs_id=true);
newds

5×9 Dataset
 Row │ xid       x1        x2        x4        y1        y2        y4        o ⋯
     │ identity  identity  identity  identity  identity  identity  identity  i ⋯
     │ Int64?    Int64?    Float64?  String?   Int64?    Int64?    String?   I ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      111         1      -1.2  abcd             3         0  abc         ⋯
   2 │      111         1      -1.2  abcd             3         0  abcd
   3 │      222         2      -3.0  efgh             3        -3  efg
   4 │      333         1       2.1  ijkl             3         2  qrst
   5 │      222         3       2.8  qrst             3        -3  efg         ⋯
                                                               2 columns omitted

## big example
1000 rows * 1000 rows

In [9]:
newds = cartesianjoin_timer(dsl_big,dsr_big,
        on = [:x1=>:y1, :x2=>:y2=>fun1, :x3=>:y3=>isless]);

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:           7.29ms /  99.7%            304KiB /  99.3%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 generate new ds            1   5.08ms   69.8%  5.08ms    242KiB   80.4%   242KiB
   2-5 newds dsr            1   3.47ms   47.8%  3.47ms   97.0KiB   32.2%  97.0KiB
     5.2 fill _res          4   3.28ms   45.1%   820μs     64.0B    0.0%    16.0B
     5.1 init _res          4   64.7μs    0.9%  16.2μs   85.2KiB   28.3%  21.3KiB
   2-1 left row count       1   1.02ms   14.0%  1.02ms   12.0KiB    4.0%  12.0KiB
   2-2 left row _res        1    545μs    7.5%   545μs    128KiB   42.4%   128KiB
     2.2 fill _

In [35]:
@time newds = cartesianjoin(dsl_big,dsr_big,
        on = [:x1=>:y1=>isless,:x2=>:y2=>isless, :x3=>:y3],check = false);
nrow(newds)

  0.010510 seconds (644 allocations: 1.237 MiB)


2351

## large example
10000 rows * 10000 rows

In [38]:
@time nds = cartesianjoin_timer(dsl_big2,dsr_big2,
              on = [:x1=>:y1, :x2=>:y2=>fun1, :x4=>:y4=>isless, :x4=>:y5=>fun2],
              check=false,obs_id=[true,true]);
nrow(nds)

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            618ms / 100.0%           21.8MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 generate new ds            1    467ms   75.6%   467ms   21.2MiB   97.1%  21.2MiB
   2-5 newds dsr            1    341ms   55.2%   341ms   2.78MiB   12.7%  2.78MiB
     5.2 fill _res          4    340ms   55.0%  84.9ms     64.0B    0.0%    16.0B
     5.1 init _res          4    793μs    0.1%   198μs   2.76MiB   12.7%   708KiB
   2-1 left row count       1   98.7ms   16.0%  98.7ms    117KiB    0.5%   117KiB
   2-2 left row _res        1   6.67ms    1.1%  6.67ms   3.49MiB   16.0%  3.49MiB
     2.2 fill _

82777

In [31]:
@benchmark cartesianjoin(dsl_big2,dsr_big2,
              on = [:x1=>:y1=>isless,:x2=>:y2=>isless,:x3=>:y3,:x4=>:y4=>isless],
              check=false,obs_id=[true,true])

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  637.344 ms … 787.125 ms  ┊ GC (min … max): 0.15% … 1.12%
 Time  (median):     660.708 ms               ┊ GC (median):    1.58%
 Time  (mean ± σ):   689.742 ms ±  60.446 ms  ┊ GC (mean ± σ):  5.21% ± 6.28%

  █▁   ▁       ▁                       ▁            ▁         ▁  
  ██▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█ ▁
  637 ms           Histogram: frequency by time          787 ms <

 Memory estimate: 121.10 MiB, allocs estimate: 9222.

In [40]:
@time nds1 = cartesianjoin(dsl_big2,dsr_big2,on = [:x1=>:y1, :x2=>:y2, :x3=>:y3],
check=false);
nrow(nds1)

  0.533631 seconds (537 allocations: 95.986 MiB, 27.18% gc time)


102

In [41]:
@time nds2 = innerjoin(dsl_big2,dsr_big2,on = [:x1=>:y1, :x2=>:y2, :x3=>:y3],check=false);
nrow(nds2)

  0.024912 seconds (1.58 k allocations: 665.000 KiB)


102

In [42]:
isequal(nds1,nds2)

true